In [ ]:
# !pip uninstall tidyzoning -y
# !pip install git+https://github.com/HOUPU1993/tidyzoning.git

Found existing installation: tidyzoning 0.1.0
Uninstalling tidyzoning-0.1.0:
  Successfully uninstalled tidyzoning-0.1.0
  Cloning https://github.com/HOUPU1993/tidyzoning.git to c:\users\admin\appdata\local\temp\pip-req-build-gpeekdpg
  Resolved https://github.com/HOUPU1993/tidyzoning.git to commit 1797a915ddef23eb7630c0352ecf8aa632ec2f2b
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tidyzoning: filename=tidyzoning-0.1.0-py3-none-any.whl size=3079 sha256=7915abbf239898924fc81b7d2963efd56488fceaff6803b43fd2a68c56d1d873
  Stored in directory: C:\Users\Admin\AppData\Local\Temp\pip-ephem-wheel-cache-q7nlxvoi\wheels\f4\29\51\e431eb95e0bf6589310375ce483daada9c916a6f65f3b65a8f
Successfully built tidyzoning


  Running command git clone --filter=blob:none --quiet https://github.com/HOUPU1993/tidyzoning.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-gpeekdpg'


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from shapely.geometry import Point
from shapely.geometry import box


from tidyzoning import find_district_idx

ModuleNotFoundError: No module named 'tidyzoning.custom_formula'

In [ ]:
tidybuilding = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data/Tidybuildings/tidybuilding_2family.geojson")
tidyparcel = gpd.read_file(r"C:/Users/Admin/Desktop/zoning_data/Test_Data/Test_Parcels/testparcel_100per_correct.geojson")
tidyzoning = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data//Tidyzoning/tidyzoning_Haltom_City.geojson")

tidyzoning = tidyzoning[tidyzoning['geometry'].notnull()]
tidyzoning = tidyzoning.to_crs(tidyparcel.crs)
tidybuilding = tidybuilding.to_crs(tidyparcel.crs)

In [106]:
tidyparcel[tidyparcel['side'] == 'centroid']

,Prop_ID,GEO_ID,parcel_id,parcel_addr,landuse,landuse_spec,parcel_label,Found_Match,match_road_address,shape_index,50_threshold,num_edges,angle_difference,shared_side,parcel_bearing,road_bearing,angle,distance_to_road,side,geometry
3,03027805,NaN,1000,3924 EARL ST,R,A,regular inside parcel,True,Earl St,1.192563,False,4,0.790292,True,-1.589407,0.011927,88.250309,29.215929,centroid,POINT (-10830185.813 3866953.859)
8,03027783,NaN,1001,3932 EARL ST,R,A,regular inside parcel,True,Earl St,1.195293,True,4,1.184516,True,-3.139301,0.011927,0.552076,51.431007,centroid,POINT (-10830141.971 3866953.571)
10,03027791,NaN,1002,3928 EARL ST,R,A,regular inside parcel,True,Earl St,1.195622,True,4,1.413652,True,1.552186,0.011927,88.250309,29.215929,centroid,POINT (-10830163.772 3866953.650)
18,00684619,NaN,1003,4100 EARL ST,R,A,regular inside parcel,True,Earl St,1.201476,True,4,1.577447,True,1.569054,-0.019660,88.973392,35.539433,centroid,POINT (-10829906.013 3866937.462)
22,06687849,NaN,1004,4101 CARNATION AVE,NaN,C1,regular inside parcel,True,Carnation Ave,1.157407,False,4,0.310866,True,-1.564429,0.006137,89.986800,66.915282,centroid,POINT (-10829950.255 3866894.020)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75858,03027848,NaN,995,3912 EARL ST,R,A,regular inside parcel,True,Earl St,1.198487,True,4,1.002128,False,-0.005563,0.011927,1.002128,6.159155,centroid,POINT (-10830252.550 3866953.973)
75863,03027856,NaN,996,3908 EARL ST,R,A,regular inside parcel,True,Earl St,1.222365,True,4,0.115853,True,-3.135434,0.011927,0.330501,49.900151,centroid,POINT (-10830273.166 3866953.848)
75868,03027821,NaN,997,3916 EARL ST,R,A,regular inside parcel,True,Earl St,1.175600,False,4,0.838519,True,-3.138975,0.011927,0.533405,50.314100,centroid,POINT (-10830229.358 3866953.944)
75871,04723295,NaN,998,3920 EARL ST,R,A,regular inside parcel,True,Earl St,1.214266,True,4,1.325428,False,-0.011206,0.011927,1.325428,6.942182,centroid,POINT (-10830207.004 3866953.865)


In [45]:
tidyzoning

,dist_info,lot_constraints,structure_constraints,other_constraints,geometry
0,"{'dist_name': 'Single-Family Residential', 'di...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...","MULTIPOLYGON (((-10829182.585 3866215.182, -10..."
1,"{'dist_name': 'Single-Family Residential', 'di...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...","MULTIPOLYGON (((-10829727.287 3866115.173, -10..."
3,"{'dist_name': 'Duplex Residential', 'dist_abbr...","{'lot_size': [{'unit': 'acres', 'use_name': ['...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...","MULTIPOLYGON (((-10829844.874 3866386.842, -10..."
6,"{'dist_name': 'Multiple Family Residential', '...",[ ],"{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...","MULTIPOLYGON (((-10827770.055 3873318.245, -10..."
7,"{'dist_name': 'Office', 'dist_abbr': 'O', 'dis...",[ ],[ ],[ ],"MULTIPOLYGON (((-10830277.539 3866854.440, -10..."
8,"{'dist_name': 'Commercial', 'dist_abbr': 'C-1'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828680.964 3866302.949, -10..."
9,"{'dist_name': 'Commercial', 'dist_abbr': 'C-2'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828928.601 3866165.418, -10..."
10,"{'dist_name': 'Commercial', 'dist_abbr': 'C-3'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828419.109 3866140.307, -10..."
11,"{'dist_name': 'Commercial', 'dist_abbr': 'C-4'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10830098.471 3866467.918, -10..."
12,"{'dist_name': 'Commercial', 'dist_abbr': 'C-5'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828352.614 3866559.640, -10..."


In [60]:
tidyzoning

,dist_info,lot_constraints,structure_constraints,other_constraints,geometry
0,"{'dist_name': 'Single-Family Residential', 'di...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...","MULTIPOLYGON (((-10829182.585 3866215.182, -10..."
1,"{'dist_name': 'Single-Family Residential', 'di...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...","MULTIPOLYGON (((-10829727.287 3866115.173, -10..."
3,"{'dist_name': 'Duplex Residential', 'dist_abbr...","{'lot_size': [{'unit': 'acres', 'use_name': ['...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...","MULTIPOLYGON (((-10829844.874 3866386.842, -10..."
6,"{'dist_name': 'Multiple Family Residential', '...",[ ],"{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...","MULTIPOLYGON (((-10827770.055 3873318.245, -10..."
7,"{'dist_name': 'Office', 'dist_abbr': 'O', 'dis...",[ ],[ ],[ ],"MULTIPOLYGON (((-10830277.539 3866854.440, -10..."
8,"{'dist_name': 'Commercial', 'dist_abbr': 'C-1'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828680.964 3866302.949, -10..."
9,"{'dist_name': 'Commercial', 'dist_abbr': 'C-2'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828928.601 3866165.418, -10..."
10,"{'dist_name': 'Commercial', 'dist_abbr': 'C-3'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828419.109 3866140.307, -10..."
11,"{'dist_name': 'Commercial', 'dist_abbr': 'C-4'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10830098.471 3866467.918, -10..."
12,"{'dist_name': 'Commercial', 'dist_abbr': 'C-5'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828352.614 3866559.640, -10..."


In [110]:
find_district_idx_results = find_district_idx(tidyparcel, tidyzoning)
find_district_idx_results = pd.DataFrame(find_district_idx_results, columns=["Prop_ID", "parcel_id", "zoning_index"])
find_district_idx_results

,Prop_ID,parcel_id,zoning_index
0,03027805,1000,1.0
1,03027783,1001,1.0
2,03027791,1002,1.0
3,00684619,1003,3.0
4,06687849,1004,3.0
...,...,...,...
15189,03027848,995,1.0
15190,03027856,996,1.0
15191,03027821,997,1.0
15192,04723295,998,1.0


In [93]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15194 entries, 0 to 15193
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Prop_ID       15194 non-null  object 
 1   parcel_id     15194 non-null  object 
 2   zoning_index  10709 non-null  float64
dtypes: float64(1), object(2)
memory usage: 356.2+ KB


In [94]:
find_district_idx_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15194 entries, 0 to 15193
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Prop_ID       15194 non-null  object 
 1   parcel_id     15194 non-null  object 
 2   zoning_index  10709 non-null  float64
dtypes: float64(1), object(2)
memory usage: 356.2+ KB
